In [1]:
import sys
sys.path.append('/Users/jsh/code/drivendata/spacecrafts/')
import shutil
from pathlib import Path
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import pytorch_lightning as L
from submission_src.spacecraft_datamodule import SpacecraftDataModule, SpacecraftDataModule, collate_fn
from submission_src.spacecraft_model import SpacecraftModel
from submission_src.MyFasterRCNN import MyFasterRCNN

In [2]:
# Set configuration variables
PROJ_DIRECTORY = Path.cwd().parent
DATA_DIRECTORY = PROJ_DIRECTORY / "data"
DEV_DIRECTORY = PROJ_DIRECTORY / "data_dev"
IMAGES_DIRECTORY = DEV_DIRECTORY / "images"
SUBMISSION_SRC_DIR = PROJ_DIRECTORY / "submission_src"
IMAGE_SIZE = (1280, 1024)
IMAGE_RESIZE_SCALE = (1/8)
IMAGE_RESIZED = (int(IMAGE_SIZE[0]*IMAGE_RESIZE_SCALE), int(IMAGE_SIZE[1]*IMAGE_RESIZE_SCALE))
VALIDATION_IDS_PATH = PROJ_DIRECTORY / 'notebooks' / 'validation_ids.csv'

In [3]:
# Import img ids for validation set used on Kaggle.
validation_ids = pd.read_csv(VALIDATION_IDS_PATH)['0'].tolist()

# Move validation set images to data/images directory.
dest_dir = DATA_DIRECTORY / "images"
if not dest_dir.exists():
    dest_dir.mkdir(parents=True, exist_ok=True)

for img in validation_ids:
    shutil.copy2(IMAGES_DIRECTORY / f"{img}.png", dest_dir / f"{img}.png")

# Create submission_format file that corresponds to the validation set.
submission_format = pd.read_csv(DEV_DIRECTORY / "submission_format.csv", index_col="image_id")
submission_format_val = submission_format.loc[validation_ids]
submission_format_val.to_csv(DATA_DIRECTORY / "submission_format.csv")

# Create train_labels file that corresponds to the validation set.
train_labels = pd.read_csv(DEV_DIRECTORY / "train_labels.csv", index_col="image_id")
test_labels = train_labels.loc[validation_ids]
test_labels.to_csv(DATA_DIRECTORY / "test_labels.csv")

# Download base model.
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(weights="DEFAULT")

# Adjust the model for spacecraft recognition task.
num_classes = 2 # 2 classes are spacecraft or background
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace pre-trained head with new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)



In [4]:
!ls ../data/images/ | wc -l

    4801


In [5]:
ckpt_path = '/Users/jsh/code/drivendata/spacecrafts/notebooks/lightning_logs/version_5/checkpoints/epoch=9-step=6570.ckpt'
checkpoint = torch.load(ckpt_path, map_location=torch.device('cpu'))

In [6]:
state_dict = checkpoint['state_dict']

In [7]:
# Load model state_dict and save full version to the submission_src folder.
fcrnn = MyFasterRCNN(model)
fcrnn.load_state_dict(state_dict)
torch.save(model, SUBMISSION_SRC_DIR / 'frcnn10epochs.pt')

In [8]:
!cd {PROJ_DIRECTORY} && make pack-submission

mkdir -p submission/
cd submission_src; zip -r ../submission/submission.zip ./*
  adding: MyFasterRCNN.py (deflated 63%)
  adding: __pycache__/ (stored 0%)
  adding: __pycache__/spacecraft_model.cpython-310.pyc (deflated 50%)
  adding: __pycache__/MyFasterRCNN.cpython-310.pyc (deflated 49%)
  adding: __pycache__/spacecraft_datamodule.cpython-310.pyc (deflated 50%)
  adding: frcnn10epochs.pt (deflated 7%)
  adding: lightning5.pt (deflated 7%)
  adding: main.py (deflated 62%)
  adding: main.sh (deflated 26%)
  adding: spacecraft_datamodule.py (deflated 67%)
  adding: spacecraft_model.py (deflated 64%)


In [9]:
! cd {PROJ_DIRECTORY} && make test-submission

mkdir -p submission/
chmod -R 0777 submission/
docker run \
		-it \
		--network none \
		--mount type=bind,source="/Users/jsh/code/drivendata/spacecrafts"/data,target=/code_execution/data,readonly \
		--mount type=bind,source="/Users/jsh/code/drivendata/spacecrafts"/submission,target=/code_execution/submission \
		--shm-size 8g \
		--name spacecraft-pose-object-detection \
		--rm \
		bc1d6eb59ba6
+ main
+ tee /code_execution/submission/log.txt
+ cd /code_execution
+ curl --silent --connect-timeout 10 --max-time 12 www.example.com
+ data_directory=/code_execution/data
+ format_filename=/code_execution/data/submission_format.csv
+ (( i=0 ))
+ (( i<=5 ))
+ t=0
+ '[' -f /code_execution/data/submission_format.csv ']'
+ echo 'found /code_execution/data/submission_format.csv after 0 seconds; data is mounted'
found /code_execution/data/submission_format.csv after 0 seconds; data is mounted
+ break
+ '[' '!' -f /code_execution/data/submission_format.csv ']'
+ expected_filename=main.sh
++ zip -s